In [5]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

from topogram_client import TopogramAPIClient

from csv import DictReader
import os
import json
import pymongo
#import musicbrainzngs
import arrow
from datetime import datetime
from pymongo import MongoClient
from collections import defaultdict
from dateutil import parser
from datetime import timedelta
import pickle

In [6]:
def create_topogram(title, nodes, edges):

    print "Creating topogram '%s'"%title

    try :
        r = topogram.create_topogram(title)
    except ValueError:
        print '> Topogram already exists'
        r = topogram.get_topogram_by_name(title)
    print r
    topogram_ID = r["data"]["_id"]
    print "topogram ID : %s"%topogram_ID

    # get and backup existing nodes and edges
    existing_nodes = topogram.get_nodes(topogram_ID)["data"]
    existing_edges = topogram.get_edges(topogram_ID)["data"]

    # clear existing graph
    if len(existing_nodes):
        topogram.delete_nodes([n["_id"] for n in existing_nodes])
        print "%s nodes deleted"%len(existing_nodes)
    if len(existing_edges):
        topogram.delete_edges([n["_id"] for n in existing_edges])
        print "%s edges deleted"%len(existing_edges)

    r = topogram.create_nodes(topogram_ID, nodes)
    print r
    print "%s nodes created."%len(r["data"])
    r = topogram.create_edges(topogram_ID, edges)
    print "%s edges created."%len(r["data"])

    print "done. Topogram has been updated. Check it at %s/topograms/%s"%(TOPOGRAM_URL, topogram_ID)



In [12]:
# setup mongo
client = MongoClient()
db = client["bandstour"]
bandsintown =  db["minedArtists"]

# credentials
TOPOGRAM_URL = "https://app.topogram.io"
USER = "gregory.bahde@laposte.net"
PASSWORD = "matrix"

# data
title = "Foo Fighters bandstour 0.1"
my_nodesdict = {}
my_nodes= []
my_edges = []


# date parsing
DATETIME_FORMAT = '%Y-%m-%dT%H:%M:%S'
DATETIME_FORMAT_FOR_MSBNZ ='%Y-%m-%d'

#retrieve trip coords for each artist
dict_coords={}

for artist in db.minedArtists.find({"totalKm": {"$gte": 1000000}}) :
 title = artist["name"]   
 if os.path.exists("./bands_dicts/"+artist["name"]+".obj")==True:
                   pass
 else:
    my_nodesdict = {}
    my_nodes= []
    my_edges = []
    my_nodesdictList=[]
    
    print artist["name"]
    #print artist
    dates_coords = []
    dates = artist.get("gigs")
    dateini = 0
    for date in dates:
        #print date.get("datetime")
        venue = date.get("venue")
        latitude = venue.get("latitude")
        longitude = venue.get("longitude")
        #print venue
        ###concat key
        idd= venue["name"]+str(int(venue["latitude"]))+str(int(venue["longitude"]))
        
        if idd not in my_nodesdict:
            node = {
                "id" : idd,
                "name" : venue["name"],
                "lat" : float(venue["latitude"]),
                "lng" : float(venue["longitude"]),
                "weight" : float(1),
                "start" : date.get("datetime").isoformat(),
                "end" : (date.get("datetime")+timedelta(days=1)).isoformat()
                }
        else:
            
            node = {
                "id" : idd,
                "name" : venue["name"],
                "lat" : float(venue["latitude"]),
                "lng" : float(venue["longitude"]),
                "weight" : float(my_nodesdict[idd]["data"].get("weight"))+1 ,
                "start" : my_nodesdict[idd]["data"].get("start"),
                "end" : date.get("datetime").isoformat()
                }
        if dateini == 0:
            dateini =1
        else:
            #print my_nodes[-1]["data"]["id"]
            edge = {
                "source" : my_nodes[-1]["data"]["id"],
                "target" : node["id"],
                "color" : "red"
#                "weight" : float(e["weight"])
            }
            #print edge
            my_edges.append({ "data" : edge })
        my_nodes.append({ "data" : node })   
        my_nodesdict[node["id"]]= { "data" : node }
    #print my_nodes
    #print "<=============================================>"
    #print my_nodesdict
    #print "<=============================================>"
    #print my_edges

        #print "coords",latitude,"/",longitude
    for key, value in my_nodesdict.iteritems():
            my_nodesdictList.append(value)
    with open("./bands_dicts/"+artist["name"]+".obj", 'wb') as handle:
                            pickle.dump([artist["name"]+"/BETA_0.2",my_nodesdictList,my_edges], handle, protocol=pickle.HIGHEST_PROTOCOL)    


Martin Garrix
Joris Voorn
Foals
Beautiful
Every Time I Die
Major Lazer
Datsik
Mo
Robbie Rivera
Mark De Clive-Lowe
The Devil Wears Prada
Bonobo
Marcel Dettmann
DJ Snake
BLOND:ISH
The Juan MacLean
Carmen
Less Than Jake
Matthew Dear
Andy C
Tiefschwarz
Stick to Your Guns
Joe Maz
Ruthie Foster
Heidi
DJ Diamond Fan Page
Yves V
Satoshi Tomiie
Chus+Ceballos
Matador
Menno de Jong
ATB
Talib Kweli
August Burns Red
Dannic
Richie Hawtin
Deadmau5
Closed
AC Slater
Jerome Isma-Ae
Host
White
Anja Schneider
KO:YU
SICK INDIVIDUALS
Crystal Castles
Suicide Silence
Joe Bonamassa
You
Justin Martin
Sold Out
The Upbeats
Carlo Lio
Boyz II Men
Kings of Leon
Dirtyphonics
A Day To Remember
Robert Glasper
Rufus Wainwright
Chris Liebing
Air Supply
PRIVATE EVENT
Showtek
Terror
DJ Politik
Morgan Page
Parkway Drive
Steel Panther
The Martinez Brothers
Dillon Francis
Joachim Garraud
Cancelled
Kryoman
Jesse Marco
Tiga
Derrick May
Alex
Nick Warren
Tommy Emmanuel
Butch
Chris Lake
Beethoven
Bryan Adams
Battle of the Bands
An

In [2]:
                     if os.path.exists("./release_mbnz/"+artist["name"]+artist["mbid"]+".obj")==True:
                      with open("./release_mbnz/"+artist["name"]+artist["mbid"]+".obj", 'rb') as handle:
                        artist["releases"]= pickle.load(handle)
                     else:
                      try:
                        artist["releases"]=(musicbrainzngs.browse_releases( artist["mbid"], release_type=["album"], includes=["labels"] ))
                        with open("./release_mbnz/"+artist["name"]+artist["mbid"]+".obj", 'wb') as handle:
                            pickle.dump(artist["releases"], handle, protocol=pickle.HIGHEST_PROTOCOL)    
                                

Foo Fighters


In [3]:
print len(my_nodesdict)

643


In [13]:
# connect to the topogram instance (pass debug=True params for more info )
topogram = TopogramAPIClient(TOPOGRAM_URL) #, debug=True)

# create a new user
try :
    topogram.create_user(USER, PASSWORD)
except ValueError:
    print "> User has already been created."

# login a new user if needed
resp_user_login = topogram.user_login(USER, PASSWORD)
print resp_user_login

assert(resp_user_login["status"] == "success")
assert(resp_user_login["status_code"] == 200)


ERROR:topogram-client:403 Unauthorized request


> User has already been created.
{u'status': u'success', 'status_code': 200, u'data': {u'authToken': u'whPM-gw0DcijurAPEycRYnvcAyB4dXSkGkirgGm1oCC', u'userId': u'DaofbRenqYpqqNmtz'}}


In [14]:
print my_nodesdictList

[{'data': {'end': '2013-04-08T19:00:00', 'name': u'Lollapalooza', 'weight': 1.0, 'start': '2013-04-07T19:00:00', 'lat': -33.437269, 'lng': -70.650564, 'id': u'Lollapalooza-33-70'}}, {'data': {'end': '2016-02-27T21:00:00', 'name': u'Life In Color', 'weight': 1.0, 'start': '2016-02-26T21:00:00', 'lat': 40.8184465, 'lng': -96.7128179, 'id': u'Life In Color40-96'}}, {'data': {'end': '2016-07-01T22:00:00', 'name': u'House Of Blues', 'weight': 1.0, 'start': '2016-06-30T22:00:00', 'lat': 28.367735, 'lng': -81.519822, 'id': u'House Of Blues28-81'}}, {'data': {'end': '2012-09-15T22:00:00', 'name': u'Kaufleuten', 'weight': 1.0, 'start': '2012-09-14T22:00:00', 'lat': 47.371722, 'lng': 8.536468, 'id': u'Kaufleuten478'}}, {'data': {'end': '2013-03-30T19:00:00', 'name': u'Hype', 'weight': 1.0, 'start': '2013-03-29T19:00:00', 'lat': -34.5875, 'lng': -58.6725, 'id': u'Hype-34-58'}}, {'data': {'end': '2016-03-17T12:00:00', 'name': u'SURFCOMBER HOTEL', 'weight': 1.0, 'start': '2016-03-16T12:00:00', 'lat

In [61]:
title = "Foo Fighters2"
create_topogram(title, my_nodesdictList, my_edges)

ERROR:topogram-client:500 - Error : A topogram with the same name already exists


Creating topogram 'Foo Fighters2'
> Topogram already exists
{u'status': u'success', 'status_code': 200, u'data': {u'sharedPublic': False, u'title': u'Foo Fighters2', u'userId': u'DaofbRenqYpqqNmtz', u'_id': u'dJHizpmPCPpeHTred', u'slug': u'foo-fighters2', u'createdAt': u'2017-10-12T12:50:51.139Z'}, u'statusCode': 200}
topogram ID : dJHizpmPCPpeHTred
{u'status': u'success', 'status_code': 200, u'data': [u'f45ovdgr2zHNNcRFb', u'byuhsGPD75CfPA42S', u'99MEgWL2kkce6t8DM', u'gReZh5cftFn5E7R8r', u'km4eZLTMg6rXBSBgc', u'ZuKiNfmYGcmqCoQ93', u'm5gqZ4f4jkyRpPh5s', u'gSXxLykyDdyFh62Lt', u'JGC4YYKS6hnkhy2Pf', u'bjj2SuNyZCH7Nc5oD', u'C9v38hrmgznWs9Kst', u'tWRmf3QzaZBGuXupE', u'rCQ7Anxzyer3hoESY', u'567nhvB9ahzcGE6hk', u'vmMsQLwJ2nw2KvKv3', u'caCciLXYxjC9LemPj', u'3dC9m85sAXjwMJsn9', u'g3jXW6nWKgDWmLSqb', u'Za4mRn9C2q6DEjJNi', u'6RwrwhLbnHxt5TXzb', u'iwNTcbg2zbDv9gXcC', u'rD6BeN6QXBPZorwsu', u'm7AZLQrgKYdxYDdY4', u'k3iZHbA5REBhKhWrH', u'tWggA6h5SfjtACiZ7', u'wDRzZZzsRgJ625dZR', u'GLured2tKmk8Q6E8S', 

In [ ]:
    #print artist
    

        dates_coords.append( [latitude,longitude])
    #print "datec",date_coords
    datapushed={}
    datapushed["dates_coords_lat_long"]=dates_coords
    datapushed["id"]=artist["_id"]
    datapushed["slug"]=artist["slug"]
    print datapushed
    dict_coords[artist["slug"]]=datapushed
print dict_coords



with open('data/test_data_nodes.csv') as f :
    reader = DictReader(f)
    for n in reader :
        node = {
            "id" : n["id"],
            "name" : n["name"],
            "lat" : float(n["lat"]),
            "lng" : float(n["lng"]),
            "weight" : float(n["weight"]),
            "start" : n["year_start"],
            "end" : n["year_stop"]
            }
        my_nodes.append({ "data" : node })

print my_nodes

with open('data/test_data_edges.csv') as f :
    reader = DictReader(f)
    for e in reader :
        edge = {
            "source" : e["source"],
            "target" : e["target"],
            "weight" : float(e["weight"])
        }
        my_edges.append({ "data" : edge })

# print my_edges

def create_topogram(title, nodes, edges):

    print "Creating topogram '%s'"%title

    try :
        r = topogram.create_topogram(title)
    except ValueError:
        print '> Topogram already exists'
        r = topogram.get_topogram_by_name(title)

    topogram_ID = r["data"]["_id"]
    print "topogram ID : %s"%topogram_ID

    # get and backup existing nodes and edges
    existing_nodes = topogram.get_nodes(topogram_ID)["data"]
    existing_edges = topogram.get_edges(topogram_ID)["data"]

    # clear existing graph
    if len(existing_nodes):
        topogram.delete_nodes([n["_id"] for n in existing_nodes])
        print "%s nodes deleted"%len(existing_nodes)
    if len(existing_edges):
        topogram.delete_edges([n["_id"] for n in existing_edges])
        print "%s edges deleted"%len(existing_edges)

    r = topogram.create_nodes(topogram_ID, nodes)
    print r
    print "%s nodes created."%len(r["data"])
    r = topogram.create_edges(topogram_ID, edges)
    print "%s edges created."%len(r["data"])

    print "done. Topogram has been updated. Check it at %s/topograms/%s"%(TOPOGRAM_URL, topogram_ID)

# connect to the topogram instance (pass debug=True params for more info )
topogram = TopogramAPIClient(TOPOGRAM_URL) #, debug=True)

# create a new user
try :
    topogram.create_user(USER, PASSWORD)
except ValueError:
    print "> User has already been created."

# login a new user if needed
resp_user_login = topogram.user_login(USER, PASSWORD)
print resp_user_login

assert(resp_user_login["status"] == "success")
assert(resp_user_login["status_code"] == 200)

create_topogram("Geo-time network", my_nodes, my_edges)